In [41]:
import numpy as np
import matplotlib.pyplot as plt
from src.analysis.load_SAIdata import open_mfdataset, Cases
import src.analysis.tracks as tracks

In [52]:
exps = {'ref':'Reference', 'rcp':'RCP8.5', 'sai':'SAI2050'}
experiment = 'ref'
run_ensemble = 1

In [3]:
if (experiment == 'ref') and (run_ensemble<=5):
    stream          = 'h1' # file stream with 3 hourly instantaneous variables
    stream_prec     = 'h2' # file stream with 3 hourly average precipitation
elif (experiment=='ref') and (run_ensemble==6):
    stream          = 'h5' # file stream with 3 hourly instantaneous variables
    stream_prec     = 'h3' # file stream with 6 hourly average precipitation
elif (experiment == 'rcp') and (run_ensemble<=5):
    stream          = 'h1' # file stream with 3 hourly instantaneous variables
    stream_prec     = 'h2' # file stream with 3 hourly average precipitation
elif (experiment=='rcp') and (run_ensemble==6):
    stream          = 'h5' # file stream with 3 hourly instantaneous variables
    stream_prec     = 'h3' # file stream with 6 hourly average precipitation
elif experiment == 'sai':
    stream          = 'h5' # file stream with 3 hourly instantaneous variables
    stream_prec     = 'h3' # file stream with 6 hourly average precipitation
else:
    print(f"unknown combination: {experiment=}, {run_ensemble=}")

In [ ]:
ds_cam = Cases('hres.ref.1').select('atm','h3').open_mfdataset()

In [30]:
ds_tracks = tracks.load_tracks('../../jobs/Tracking_TC_RV.infext2/', ext='.infext')

REF.001: 5.000 years, 470 tracks
REF.002: 5.000 years, 480 tracks
REF.003: 5.000 years, 490 tracks
REF.004: 5.000 years, 493 tracks
REF.005: 5.000 years, 514 tracks
REF.006: 5.000 years, 454 tracks
RCP.001: 5.000 years, 440 tracks
RCP.002: 5.000 years, 439 tracks
RCP.003: 5.000 years, 457 tracks
RCP.004: 5.000 years, 458 tracks
RCP.005: 5.000 years, 451 tracks
RCP.006: 5.000 years, 468 tracks
SAI.001: 5.000 years, 422 tracks
SAI.002: 5.000 years, 436 tracks
SAI.003: 5.000 years, 407 tracks
SAI.004: 5.000 years, 428 tracks
SAI.005: 5.000 years, 409 tracks
SAI.006: 5.000 years, 413 tracks


In [62]:
# for each year and ensemble member, create array of timestamps at which TCs are active
track_times = {}
for exp in exps:
    Exp = exps[exp]
    for ens in np.unique(ds_tracks[Exp].ens):
        ds_expens = ds_tracks[Exp].where(ds_tracks[Exp].ens==ens, drop=True)
        for year in np.unique(ds_expens.year):
            if np.isnan(year):
                continue
            dsi = ds_expens.where(ds_expens.year==year, drop=True)
            track_times_i = np.sort(np.unique(dsi.time))
            track_times_i = track_times_i[~np.isnan(track_times_i)]
            track_times[f'{exp}.{int(ens)}.{int(year)}'] = track_times_i
#             print(exp, int(ens), int(year), len(track_times_i)/8) # days with active TCs

In [67]:
def circular_mask(r, lats, lons):
    """Return mask which is True within r degrees of TC center"""
    Nlons = lons.size # number of longitudes (total)
    Nlats = lats.size # number of latitudes (total)
    dx = round(r*Nlons/360)
    dy = round(r*Nlats/180)
    circlemask = np.sqrt((360/Nlons*np.arange(-dx,dx+1)[None,:])**2 
        + (180/Nlats*np.arange(-dy,dy+1)[:,None])**2) <= r
    return circlemask


def add_track_data(da, track, mask, result):
    """fetch data around TC center for whole track"""
    track = track.where(track.notnull(), drop=True)
    da = da.interp(time=track.cftime.data).load()
    dx = int((mask.shape[1]-1)/2)
    dy = int((mask.shape[0]-1)/2)
    for t,(lat,lon) in enumerate(zip(track.lat, track.lon)):
        x = np.searchsorted(da.lon, lon)
        y = np.searchsorted(da.lat, lat)
        xs = np.arange(x-dx,x+dx+1,)%len(da.lon)
        ys = np.arange(max(y-dy,0),min(y+dy+1,len(da.lat)))
        result[ys,xs] += da.isel(time=t)[ys,xs] * mask * 3

In [95]:
ds_tracks['Reference'].where((ds_tracks['Reference'].isel(dtime=0).ens==1) & (ds_tracks['Reference'].isel(dtime=0).year==2004),drop=True).time.isel(dtime=0)

IndexError: dimension coordinate 'dtime' conflicts between indexed and indexing objects:
<xarray.DataArray 'dtime' (dtime: 436)> Size: 3kB
array([ 0.   ,  0.125,  0.25 , ..., 54.125, 54.25 , 54.375], shape=(436,))
Coordinates:
  * dtime    (dtime) float64 3kB 0.0 0.125 0.25 0.375 ... 54.0 54.12 54.25 54.38
Attributes:
    long_name:  Lifetime TC
    units:      days
vs.
<xarray.Variable ()> Size: 8B
array(0.)
Attributes:
    long_name:  Lifetime TC
    units:      days

In [107]:
ids = (ds_tracks['Reference'].isel(dtime=0).ens == 1) & (ds_tracks['Reference'].isel(dtime=0).year == 2004)
dsi = ds_tracks['Reference'].isel(id=ids.data)
dsi

<xarray.Dataset> Size: 9MB
Dimensions:    (id: 96, dtime: 436, data: 13)
Coordinates:
  * id         (id) float64 768B 86.0 87.0 88.0 89.0 ... 178.0 179.0 182.0 185.0
  * dtime      (dtime) float64 3kB 0.0 0.125 0.25 0.375 ... 54.12 54.25 54.38
  * data       (data) float64 104B 0.0 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0 12.0
    ens        (id) float64 768B 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
Data variables: (12/16)
    TC_tracks  (id, dtime, data) float64 4MB 722.8 44.69 -15.84 ... nan nan nan
    num_days   int64 8B 10950
    time       (id, dtime) float64 335kB 722.8 722.9 723.0 723.1 ... nan nan nan
    lon        (id, dtime) float64 335kB 44.69 43.75 43.75 43.75 ... nan nan nan
    lat        (id, dtime) float64 335kB -15.84 -15.61 -15.37 ... nan nan nan
    PSL        (id, dtime) float64 335kB 1.003e+03 998.7 995.3 ... nan nan nan
    ...         ...
    SSTmon     (id, dtime) float64 335kB 28.43 28.28 28.19 28.38 ... nan nan nan
    RV         (id, dtime) float64 335kB 0.0007087 0.0007549 ... nan nan
    Vshear     (id, dtime) float64 335kB 5.298 5.315 6.719 8.82 ... nan nan nan
    PRECT      (id, dtime) float64 335kB 27.38 22.12 26.93 31.73 ... nan nan nan
    TEMP       (id, dtime) float64 335kB 0.2401 1.747 4.117 3.72 ... nan nan nan
    year       (id, dtime) float64 335kB 2.004e+03 2.004e+03 ... nan nan

In [145]:
# for each time step, loop through active TCs 
#for time in np.arange(722.75,1107.375,0.125):
for time in np.arange(722.75,740.375,0.125):
    ids = np.nonzero(dsi.time == time)
    for (tid,dt) in ids.T:
        dsii = dsi.isel(id=tid, dtime=dt)
        assert dsii.time.data == time, f'error: {time=:.3f} does not match track time {dsii.time.data:.3f} ({tid=:.0f}, {dt=:.0f})'
        print(time, dsii.lon.data, dsii.lat.data)

722.75 44.6875 -15.840938722294652
722.875 43.75 -15.606258148631028
723.0 43.75 -15.371577574967404
723.125 43.75 -15.13689700130378
723.25 43.75 -15.13689700130378
723.375 43.4375 -14.902216427640155
723.5 43.4375 -14.902216427640155
723.625 43.125 -14.902216427640155
723.75 42.5 -14.902216427640155
723.875 42.5 -14.902216427640155
724.0 42.1875 -14.902216427640155
724.125 41.875 -15.371577574967404
724.25 41.25 -15.606258148631028
724.375 40.625 -15.840938722294652
724.5 40.0 -16.075619295958276
724.625 39.0625 -16.075619295958276
724.75 38.4375 -15.371577574967404
727.875 66.5625 -10.208604954367658
728.0 66.5625 -10.208604954367658
728.125 185.625 -17.718383311603645
728.125 66.5625 -10.208604954367658
728.25 185.625 -17.95306388526727
728.25 66.5625 -9.973924380704034
728.375 185.625 -18.187744458930894
728.375 66.5625 -10.208604954367658
728.5 185.625 -18.187744458930894
728.5 66.5625 -10.208604954367658
728.625 185.625 -18.422425032594518
728.625 66.5625 -10.208604954367658
728

In [128]:
dsi.time.min(), dsi.time.max()

(<xarray.DataArray 'time' ()> Size: 8B
 array(722.75),
 <xarray.DataArray 'time' ()> Size: 8B
 array(1107.25))